In [1]:
!pip install darts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.3/917.3 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.5/869.5 kB 20.3 MB/s eta 0:00:

In [2]:
from darts import TimeSeries
from google.colab import files
import pandas as pd
import io
import matplotlib.pyplot as plt

In [57]:
uploaded = files.upload()

Saving consolidated_final_data_russia_SER.xlsx to consolidated_final_data_russia_SER.xlsx


In [59]:
df = pd.read_excel(io.BytesIO(uploaded['consolidated_final_data_russia_SER.xlsx']))
df = df.drop(df.columns[[0,2,3,4,5,7,8,11]], axis = 1)
df.head()

,spot_ER_Russia,Oil_price_growth_rate_WTI,SR_Interest_rate_diff_R_U,global_EPU(PPP),US_EMV,US_MPU
0,5.629,33.320298,42.75,76.652955,21.0498,83.390495
1,5.676,16.444206,36.81,80.081169,19.4810,87.818336
2,5.726,-1.696391,36.61,65.375401,22.9562,91.088722
3,5.762,-16.217887,30.49,74.742484,23.8621,79.398842
4,5.773,-1.939211,30.50,73.520737,17.6622,117.052849


In [60]:
cases = df[df.columns[[0]]]
covariates = df.drop(df.columns[[0]], axis = 1)

In [61]:
from darts.utils.timeseries_generation import datetime_attribute_timeseries

cases = TimeSeries.from_dataframe(cases)
type(cases)

covariates = TimeSeries.from_dataframe(covariates)
type(covariates)

darts.timeseries.TimeSeries

In [8]:
from darts.models import TransformerModel, NBEATSModel, NHiTSModel, TCNModel, RNNModel, BlockRNNModel, DLinearModel, NLinearModel, TiDEModel, TFTModel, TSMixerModel

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Horizon 1: Covariates

In [62]:
n = 1
train, test = cases[:-n], cases[-n:]
train_cvt, test_cvt = covariates[:-n], covariates[-n:]

In [63]:
DeepAR = RNNModel(input_chunk_length = 10, output_chunk_length = 13, model='RNN')
DeepAR.fit(train, epochs = 100, verbose = True)
pred_DeepAR = DeepAR.predict(1)
pred_DeepAR.to_csv('DeepAR_1.csv', encoding = 'utf-8-sig')
files.download('DeepAR_1.csv')

#LSTM = BlockRNNModel(input_chunk_length = 30, output_chunk_length = 13, n_rnn_layers = 2)
#LSTM.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_LSTM = LSTM.predict(1)
#pred_LSTM.to_csv('LSTM_1.csv', encoding = 'utf-8-sig')
#files.download('LSTM_1.csv')

NBEATS = NBEATSModel(input_chunk_length = 30, output_chunk_length = 13)
NBEATS.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_NBEATS = NBEATS.predict(1)
pred_NBEATS.to_csv('NBEATS_1.csv', encoding = 'utf-8-sig')
files.download('NBEATS_1.csv')

NHiTS = NHiTSModel(input_chunk_length = 30, output_chunk_length = 13)
NHiTS.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_NHiTS = NHiTS.predict(1)
pred_NHiTS.to_csv('NHiTS_1.csv', encoding = 'utf-8-sig')
files.download('NHiTS_1.csv')

#Transformer = TransformerModel(input_chunk_length = 30, output_chunk_length = 13)
#Transformer.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_Transformer = Transformer.predict(1)
#pred_Transformer.to_csv('Transformer_1.csv', encoding = 'utf-8-sig')
#files.download('Transformer_1.csv')

#TCN = TCNModel(input_chunk_length = 30, output_chunk_length = 13)
#TCN.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_TCN = TCN.predict(1)
#pred_TCN.to_csv('TCN_1.csv', encoding = 'utf-8-sig')
#files.download('TCN_1.csv')

NLinear = NLinearModel(input_chunk_length = 30, output_chunk_length = 13)
NLinear.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_NLinear = NLinear.predict(1)
pred_NLinear.to_csv('NLinear_1.csv', encoding = 'utf-8-sig')
files.download('NLinear_1.csv')

DLinear = DLinearModel(input_chunk_length = 30, output_chunk_length = 13)
DLinear.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_DLinear = DLinear.predict(1)
pred_DLinear.to_csv('DLinear_1.csv', encoding = 'utf-8-sig')
files.download('DLinear_1.csv')

#TiDE = TiDEModel(input_chunk_length = 30, output_chunk_length = 13)
#TiDE.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_TiDE = TiDE.predict(1)
#pred_TiDE.to_csv('TiDE_1.csv', encoding = 'utf-8-sig')
#files.download('TiDE_1.csv')

TSMixer = TSMixerModel(input_chunk_length = 30, output_chunk_length = 13)
TSMixer.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_TSMixer = TSMixer.predict(1)
pred_TSMixer.to_csv('TSMixer_1.csv', encoding = 'utf-8-sig')
files.download('TSMixer_1.csv')

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | rnn             | RNN              | 700    | train
6 | V               | Linear           | 26     | train
-------------------------------------------------------------
726       Trainable params
0         Non-trainable params
726       Total params
0.003     Total estimated model params size (MB)
7         

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

...

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Horizon 3: Covariates

In [64]:
n = 3
train, test = cases[:-n], cases[-n:]
train_cvt, test_cvt = covariates[:-n], covariates[-n:]

In [65]:
DeepAR = RNNModel(input_chunk_length = 10, output_chunk_length = 13, model='RNN')
DeepAR.fit(train, epochs = 100, verbose = True)
pred_DeepAR = DeepAR.predict(3)
pred_DeepAR.to_csv('DeepAR_3.csv', encoding = 'utf-8-sig')
files.download('DeepAR_3.csv')

#LSTM = BlockRNNModel(input_chunk_length = 30, output_chunk_length = 13, n_rnn_layers = 2)
#LSTM.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_LSTM = LSTM.predict(3)
#pred_LSTM.to_csv('LSTM_3.csv', encoding = 'utf-8-sig')
#files.download('LSTM_3.csv')

NBEATS = NBEATSModel(input_chunk_length = 30, output_chunk_length = 13)
NBEATS.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_NBEATS = NBEATS.predict(3)
pred_NBEATS.to_csv('NBEATS_3.csv', encoding = 'utf-8-sig')
files.download('NBEATS_3.csv')

NHiTS = NHiTSModel(input_chunk_length = 30, output_chunk_length = 13)
NHiTS.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_NHiTS = NHiTS.predict(3)
pred_NHiTS.to_csv('NHiTS_3.csv', encoding = 'utf-8-sig')
files.download('NHiTS_3.csv')

#Transformer = TransformerModel(input_chunk_length = 30, output_chunk_length = 13)
#Transformer.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_Transformer = Transformer.predict(3)
#pred_Transformer.to_csv('Transformer_3.csv', encoding = 'utf-8-sig')
#files.download('Transformer_3.csv')

#TCN = TCNModel(input_chunk_length = 30, output_chunk_length = 13)
#TCN.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_TCN = TCN.predict(3)
#pred_TCN.to_csv('TCN_3.csv', encoding = 'utf-8-sig')
#files.download('TCN_3.csv')

NLinear = NLinearModel(input_chunk_length = 30, output_chunk_length = 13)
NLinear.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_NLinear = NLinear.predict(3)
pred_NLinear.to_csv('NLinear_3.csv', encoding = 'utf-8-sig')
files.download('NLinear_3.csv')

DLinear = DLinearModel(input_chunk_length = 30, output_chunk_length = 13)
DLinear.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_DLinear = DLinear.predict(3)
pred_DLinear.to_csv('DLinear_3.csv', encoding = 'utf-8-sig')
files.download('DLinear_3.csv')

#TiDE = TiDEModel(input_chunk_length = 30, output_chunk_length = 13)
#TiDE.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_TiDE = TiDE.predict(3)
#pred_TiDE.to_csv('TiDE_3.csv', encoding = 'utf-8-sig')
#files.download('TiDE_3.csv')

TSMixer = TSMixerModel(input_chunk_length = 30, output_chunk_length = 13)
TSMixer.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_TSMixer = TSMixer.predict(3)
pred_TSMixer.to_csv('TSMixer_3.csv', encoding = 'utf-8-sig')
files.download('TSMixer_3.csv')

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | rnn             | RNN              | 700    | train
6 | V               | Linear           | 26     | train
-------------------------------------------------------------
726       Trainable params
0         Non-trainable params
726       Total params
0.003     Total estimated model params size (MB)
7         

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

...

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Horizon 6: Covariates

In [66]:
n = 6
train, test = cases[:-n], cases[-n:]
train_cvt, test_cvt = covariates[:-n], covariates[-n:]

In [67]:
DeepAR = RNNModel(input_chunk_length = 10, output_chunk_length = 13, model='RNN')
DeepAR.fit(train, epochs = 100, verbose = True)
pred_DeepAR = DeepAR.predict(6)
pred_DeepAR.to_csv('DeepAR_6.csv', encoding = 'utf-8-sig')
files.download('DeepAR_6.csv')

#LSTM = BlockRNNModel(input_chunk_length = 30, output_chunk_length = 13, n_rnn_layers = 2)
#LSTM.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_LSTM = LSTM.predict(6)
#pred_LSTM.to_csv('LSTM_6.csv', encoding = 'utf-8-sig')
#files.download('LSTM_6.csv')

NBEATS = NBEATSModel(input_chunk_length = 30, output_chunk_length = 13)
NBEATS.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_NBEATS = NBEATS.predict(6)
pred_NBEATS.to_csv('NBEATS_6.csv', encoding = 'utf-8-sig')
files.download('NBEATS_6.csv')

NHiTS = NHiTSModel(input_chunk_length = 30, output_chunk_length = 13)
NHiTS.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_NHiTS = NHiTS.predict(6)
pred_NHiTS.to_csv('NHiTS_6.csv', encoding = 'utf-8-sig')
files.download('NHiTS_6.csv')

#Transformer = TransformerModel(input_chunk_length = 30, output_chunk_length = 13)
#Transformer.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_Transformer = Transformer.predict(6)
#pred_Transformer.to_csv('Transformer_6.csv', encoding = 'utf-8-sig')
#files.download('Transformer_6.csv')

TCN = TCNModel(input_chunk_length = 30, output_chunk_length = 13)
TCN.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_TCN = TCN.predict(6)
pred_TCN.to_csv('TCN_6.csv', encoding = 'utf-8-sig')
files.download('TCN_6.csv')

NLinear = NLinearModel(input_chunk_length = 30, output_chunk_length = 13)
NLinear.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_NLinear = NLinear.predict(6)
pred_NLinear.to_csv('NLinear_6.csv', encoding = 'utf-8-sig')
files.download('NLinear_6.csv')

DLinear = DLinearModel(input_chunk_length = 30, output_chunk_length = 13)
DLinear.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_DLinear = DLinear.predict(6)
pred_DLinear.to_csv('DLinear_6.csv', encoding = 'utf-8-sig')
files.download('DLinear_6.csv')

#TiDE = TiDEModel(input_chunk_length = 30, output_chunk_length = 13)
#TiDE.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_TiDE = TiDE.predict(6)
#pred_TiDE.to_csv('TiDE_6.csv', encoding = 'utf-8-sig')
#files.download('TiDE_6.csv')

TSMixer = TSMixerModel(input_chunk_length = 30, output_chunk_length = 13)
TSMixer.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_TSMixer = TSMixer.predict(6)
pred_TSMixer.to_csv('TSMixer_6.csv', encoding = 'utf-8-sig')
files.download('TSMixer_6.csv')

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | rnn             | RNN              | 700    | train
6 | V               | Linear           | 26     | train
-------------------------------------------------------------
726       Trainable params
0         Non-trainable params
726       Total params
0.003     Total estimated model params size (MB)
7         

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

...

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Horizon 12: Covariates

In [68]:
n = 12
train, test = cases[:-n], cases[-n:]
train_cvt, test_cvt = covariates[:-n], covariates[-n:]

In [69]:
DeepAR = RNNModel(input_chunk_length = 10, output_chunk_length = 13, model='RNN')
DeepAR.fit(train, epochs = 100, verbose = True)
pred_DeepAR = DeepAR.predict(12)
pred_DeepAR.to_csv('DeepAR_12.csv', encoding = 'utf-8-sig')
files.download('DeepAR_12.csv')

#LSTM = BlockRNNModel(input_chunk_length = 30, output_chunk_length = 13, n_rnn_layers = 2)
#LSTM.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_LSTM = LSTM.predict(12)
#pred_LSTM.to_csv('LSTM_12.csv', encoding = 'utf-8-sig')
#files.download('LSTM_12.csv')

NBEATS = NBEATSModel(input_chunk_length = 30, output_chunk_length = 13)
NBEATS.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_NBEATS = NBEATS.predict(12)
pred_NBEATS.to_csv('NBEATS_12.csv', encoding = 'utf-8-sig')
files.download('NBEATS_12.csv')

NHiTS = NHiTSModel(input_chunk_length = 30, output_chunk_length = 13)
NHiTS.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_NHiTS = NHiTS.predict(12)
pred_NHiTS.to_csv('NHiTS_12.csv', encoding = 'utf-8-sig')
files.download('NHiTS_12.csv')

#Transformer = TransformerModel(input_chunk_length = 30, output_chunk_length = 13)
#Transformer.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_Transformer = Transformer.predict(12)
#pred_Transformer.to_csv('Transformer_12.csv', encoding = 'utf-8-sig')
#files.download('Transformer_12.csv')

#TCN = TCNModel(input_chunk_length = 30, output_chunk_length = 13)
#TCN.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_TCN = TCN.predict(12)
#pred_TCN.to_csv('TCN_12.csv', encoding = 'utf-8-sig')
#files.download('TCN_12.csv')

NLinear = NLinearModel(input_chunk_length = 30, output_chunk_length = 13)
NLinear.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_NLinear = NLinear.predict(12)
pred_NLinear.to_csv('NLinear_12.csv', encoding = 'utf-8-sig')
files.download('NLinear_12.csv')

DLinear = DLinearModel(input_chunk_length = 30, output_chunk_length = 13)
DLinear.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_DLinear = DLinear.predict(12)
pred_DLinear.to_csv('DLinear_12.csv', encoding = 'utf-8-sig')
files.download('DLinear_12.csv')

#TiDE = TiDEModel(input_chunk_length = 30, output_chunk_length = 13)
#TiDE.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_TiDE = TiDE.predict(12)
#pred_TiDE.to_csv('TiDE_12.csv', encoding = 'utf-8-sig')
#files.download('TiDE_12.csv')

TSMixer = TSMixerModel(input_chunk_length = 30, output_chunk_length = 13)
TSMixer.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_TSMixer = TSMixer.predict(12)
pred_TSMixer.to_csv('TSMixer_12.csv', encoding = 'utf-8-sig')
files.download('TSMixer_12.csv')

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | rnn             | RNN              | 700    | train
6 | V               | Linear           | 26     | train
-------------------------------------------------------------
726       Trainable params
0         Non-trainable params
726       Total params
0.003     Total estimated model params size (MB)
7         

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

...

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Horizon 24: Covariates

In [70]:
n = 24
train, test = cases[:-n], cases[-n:]
train_cvt, test_cvt = covariates[:-n], covariates[-n:]

In [71]:
DeepAR = RNNModel(input_chunk_length = 10, output_chunk_length = 25, model='RNN')
DeepAR.fit(train, epochs = 100, verbose = True)
pred_DeepAR = DeepAR.predict(24)
pred_DeepAR.to_csv('DeepAR_24.csv', encoding = 'utf-8-sig')
files.download('DeepAR_24.csv')

#LSTM = BlockRNNModel(input_chunk_length = 30, output_chunk_length = 25, n_rnn_layers = 2)
#LSTM.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_LSTM = LSTM.predict(24)
#pred_LSTM.to_csv('LSTM_24.csv', encoding = 'utf-8-sig')
#files.download('LSTM_24.csv')

NBEATS = NBEATSModel(input_chunk_length = 30, output_chunk_length = 25)
NBEATS.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_NBEATS = NBEATS.predict(24)
pred_NBEATS.to_csv('NBEATS_24.csv', encoding = 'utf-8-sig')
files.download('NBEATS_24.csv')

NHiTS = NHiTSModel(input_chunk_length = 30, output_chunk_length = 25)
NHiTS.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_NHiTS = NHiTS.predict(24)
pred_NHiTS.to_csv('NHiTS_24.csv', encoding = 'utf-8-sig')
files.download('NHiTS_24.csv')

#Transformer = TransformerModel(input_chunk_length = 30, output_chunk_length = 25)
#Transformer.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_Transformer = Transformer.predict(24)
#pred_Transformer.to_csv('Transformer_24.csv', encoding = 'utf-8-sig')
#files.download('Transformer_24.csv')

#TCN = TCNModel(input_chunk_length = 30, output_chunk_length = 25)
#TCN.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_TCN = TCN.predict(24)
#pred_TCN.to_csv('TCN_24.csv', encoding = 'utf-8-sig')
#files.download('TCN_24.csv')

NLinear = NLinearModel(input_chunk_length = 30, output_chunk_length = 25)
NLinear.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_NLinear = NLinear.predict(24)
pred_NLinear.to_csv('NLinear_24.csv', encoding = 'utf-8-sig')
files.download('NLinear_24.csv')

DLinear = DLinearModel(input_chunk_length = 30, output_chunk_length = 25)
DLinear.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_DLinear = DLinear.predict(24)
pred_DLinear.to_csv('DLinear_24.csv', encoding = 'utf-8-sig')
files.download('DLinear_24.csv')

#TiDE = TiDEModel(input_chunk_length = 30, output_chunk_length = 25)
#TiDE.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_TiDE = TiDE.predict(24)
#pred_TiDE.to_csv('TiDE_24.csv', encoding = 'utf-8-sig')
#files.download('TiDE_24.csv')

TSMixer = TSMixerModel(input_chunk_length = 30, output_chunk_length = 25)
TSMixer.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_TSMixer = TSMixer.predict(24)
pred_TSMixer.to_csv('TSMixer_24.csv', encoding = 'utf-8-sig')
files.download('TSMixer_24.csv')

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | rnn             | RNN              | 700    | train
6 | V               | Linear           | 26     | train
-------------------------------------------------------------
726       Trainable params
0         Non-trainable params
726       Total params
0.003     Total estimated model params size (MB)
7         

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

...

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Horizon 48: Covariates

In [72]:
n = 48
train, test = cases[:-n], cases[-n:]
train_cvt, test_cvt = covariates[:-n], covariates[-n:]

In [73]:
DeepAR = RNNModel(input_chunk_length = 10, output_chunk_length = 50, model='RNN')
DeepAR.fit(train, epochs = 100, verbose = True)
pred_DeepAR = DeepAR.predict(48)
pred_DeepAR.to_csv('DeepAR_48.csv', encoding = 'utf-8-sig')
files.download('DeepAR_48.csv')

#LSTM = BlockRNNModel(input_chunk_length = 10, output_chunk_length = 50, n_rnn_layers = 2)
#LSTM.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_LSTM = LSTM.predict(48)
#pred_LSTM.to_csv('LSTM_48.csv', encoding = 'utf-8-sig')
#files.download('LSTM_48.csv')

NBEATS = NBEATSModel(input_chunk_length = 10, output_chunk_length = 50)
NBEATS.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_NBEATS = NBEATS.predict(48)
pred_NBEATS.to_csv('NBEATS_48.csv', encoding = 'utf-8-sig')
files.download('NBEATS_48.csv')

NHiTS = NHiTSModel(input_chunk_length = 10, output_chunk_length = 50)
NHiTS.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_NHiTS = NHiTS.predict(48)
pred_NHiTS.to_csv('NHiTS_48.csv', encoding = 'utf-8-sig')
files.download('NHiTS_48.csv')

#Transformer = TransformerModel(input_chunk_length = 10, output_chunk_length = 50)
#Transformer.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_Transformer = Transformer.predict(48)
#pred_Transformer.to_csv('Transformer_48.csv', encoding = 'utf-8-sig')
#files.download('Transformer_48.csv')

#TCN = TCNModel(input_chunk_length = 10, output_chunk_length = 50)
#TCN.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_TCN = TCN.predict(48)
#pred_TCN.to_csv('TCN_48.csv', encoding = 'utf-8-sig')
#files.download('TCN_48.csv')

NLinear = NLinearModel(input_chunk_length = 10, output_chunk_length = 50)
NLinear.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_NLinear = NLinear.predict(48)
pred_NLinear.to_csv('NLinear_48.csv', encoding = 'utf-8-sig')
files.download('NLinear_48.csv')

DLinear = DLinearModel(input_chunk_length = 10, output_chunk_length = 50)
DLinear.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_DLinear = DLinear.predict(48)
pred_DLinear.to_csv('DLinear_48.csv', encoding = 'utf-8-sig')
files.download('DLinear_48.csv')

#TiDE = TiDEModel(input_chunk_length = 10, output_chunk_length = 50)
#TiDE.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
#pred_TiDE = TiDE.predict(48)
#pred_TiDE.to_csv('TiDE_48.csv', encoding = 'utf-8-sig')
#files.download('TiDE_48.csv')

TSMixer = TSMixerModel(input_chunk_length = 10, output_chunk_length = 50)
TSMixer.fit(train, past_covariates = train_cvt, epochs = 100, verbose = True)
pred_TSMixer = TSMixer.predict(48)
pred_TSMixer.to_csv('TSMixer_48.csv', encoding = 'utf-8-sig')
files.download('TSMixer_48.csv')

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | rnn             | RNN              | 700    | train
6 | V               | Linear           | 26     | train
-------------------------------------------------------------
726       Trainable params
0         Non-trainable params
726       Total params
0.003     Total estimated model params size (MB)
7         

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

...

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>